In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as c
import astropy.units as u
from scipy.interpolate import interp2d
from scipy.optimize import curve_fit
from scipy.optimize import minimize

import emcee
import corner

import twopoppy
import dsharp_opac as op

from dipsy_functions import get_powerlaw_dust_distribution
from dipsy_functions import bplanck

au      = c.au.cgs.value
year    = (1*u.year).cgs.value
c_light = c.c.cgs.value
jy_sas  = (1 * u.Jy / u.arcsec**2).cgs.value

%matplotlib inline

## Load opacities

In [ ]:
with np.load(op.get_datafile('default_opacities.npz')) as f:
    a_op      = f['a']
    lam_op    = f['lam']
    k_abs     = f['k_abs']
    k_sca     = f['k_sca']
    rho_s_op  = f['rho_s']

## Set up simulation

In [ ]:
args = twopoppy.args()

# make sure we use the same grain density as in the opacities

args.rhos = rho_s_op[()]

In [ ]:
args.print_args()

In [ ]:
res = twopoppy.wrapper.model_wrapper(args)

In [ ]:
# set some time snapshot index
it = -1

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.a_fr[it, :], label='fragmentation')
ax.loglog(res.x / au, res.a_dr[it, :], label='drift')
ax.loglog(res.x / au, res.a_t[it, :], 'k--', label='$a_\mathrm{max}$')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.sigma_g[it, :], label='gas')
ax.loglog(res.x / au, res.sigma_d[it, :], label='dust')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

## Plot size distribution from the code

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, res.a, np.log10(res.sig_sol), vmin=-10, vmax=1)
ax.set_xscale('log')
ax.set_yscale('log')
plt.colorbar(cc);

## Create power-law size distribution

In [ ]:
a, a_i, sig_da = get_powerlaw_dust_distribution(res.sigma_d[it, :], res.a_t[it, :], a0=args.a0, na=100)

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, a_i, np.log10(sig_da.T), vmin=-10, vmax=1)#, edgecolor='k')
ax.loglog(res.x / au, res.a_t[it, :], 'r')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(a[[0, -1]])
#ax.set_xlim(2e-1, 4e-1)
#ax.set_ylim(6e0, 2e1)
plt.colorbar(cc);

## Calculate Intensity profiles

In [ ]:
# set wavelengths

lam_obs = [0.087, 0.1, 0.3, 1]

In [ ]:
n_lam = len(lam_obs)

I_nu = np.zeros([n_lam, len(res.x)])
tau  = np.zeros([n_lam, len(res.x)])
k_a  = np.zeros([n_lam, len(a)])

for ilam, _lam in enumerate(lam_obs):
    nu_obs = c_light/_lam

    # interpolate on our wavelength and size grid

    f_interp = interp2d(np.log10(lam_op), np.log10(a_op), np.log10(k_abs))
    k_a[ilam, :] = 10.**f_interp(np.log10(_lam), np.log10(a))[:, 0]

    ## Calculate intensity profile

    tau[ilam, :]  = (sig_da * k_a[ilam,:].T).sum(-1)
    I_nu[ilam, :] = bplanck(nu_obs, res.T) * (1 - np.exp(-tau[ilam, :]))

In [ ]:
f, ax = plt.subplots()
for _lam, _Inu in zip(lam_obs, I_nu):
    ax.loglog(res.x / au, _Inu/jy_sas, label=f'$\lambda = {_lam * 10:.2g}$ mm')

ax.legend()
ax.set_ylim(1e-2, 1e2);

## Fitting the dust line

In [ ]:
def truncated_powerlaw(x, y0, p, xout, noise):
    return y0 * (x/x[0])**p * (x < xout) + noise

In [ ]:
def lnprob(params, x, data):
    
    if params[0]<1e-50:
        return -1e300
    if np.abs(params[1])>10:
        return -1e300
    if params[3]<1e-50:
        return -1e300
    
    model = truncated_powerlaw(x, *params)
    rmsd = (data - model)**2
    # we ignore points that are too far away from the model
    #rmsd[rmsd>(100 * noise)**2] = 0.0
    rmsd /= 2 * noise**2

    return -rmsd.sum()

In [ ]:
noise = 1e-3 * jy_sas
x     = np.linspace(res.x[0], 1e3 * au, 100)
data  = np.interp(x, res.x, I_nu[0, :] + noise * np.random.randn(args.nr))
p0    = [data[0], -0.5, 100 * au, noise]

In [ ]:
nwalkers = 40
nburnin = 200
nsteps = 6000
sampler = emcee.EnsembleSampler(nwalkers, len(p0), lnprob, args=[x, data])

inisamples = np.array([
    p0[0] * 10**(-2 + 4 * np.random.rand(nwalkers)),
    p0[1] + (-1 + 2 * np.random.rand(nwalkers)),
    200 * au * np.random.rand(nwalkers),
    noise * 10**(-2 + np.random.rand(nwalkers))]).T

# first burn in to keep the ones with reasonable acceptance fraction

burnin = sampler.run_mcmc(inisamples, nburnin)
good = inisamples[sampler.acceptance_fraction>0.25, :]
inisamples = good[np.random.choice(np.arange(len(good)), size=nwalkers)]

# second burn in to keep the ones with higher probability

sampler.reset()
burnin = sampler.run_mcmc(inisamples, nburnin)
final_prob = sampler.lnprobability[:, -1]
good = np.arange(nwalkers)[final_prob > np.sort(final_prob)[nwalkers//2]]
inisamples = inisamples[np.random.choice(good, size=nwalkers)]

sampler.reset()
output = sampler.run_mcmc(inisamples, nsteps)

Check convergence

In [ ]:
plt.loglog(np.arange(nsteps), -sampler.lnprobability.T);

In [ ]:
acc_time = sampler.get_autocorr_time()

In [ ]:
discard = int(5 * acc_time.max())

In [ ]:
flat_chain = sampler.chain[:, discard:, :].reshape(-1, 4)

In [ ]:
f, ax = plt.subplots()
ax.loglog(x / au, data, label='data')
ax.loglog(x / au, truncated_powerlaw(x, *p0), 'k--', label='guess')
for sample in flat_chain[-10:,:]:
    ax.loglog(x / au, truncated_powerlaw(x, *sample), lw=0.5, alpha=0.5)

ax.set_ylim(1e-16, 1e-10)
ax.legend();

In [ ]:
corner_data = flat_chain.copy()

corner_data[:, 0] = np.log10(corner_data[:, 0])
corner_data[:, 2] /= au
corner_data[:, 3] = np.log10(corner_data[:, 3])

In [ ]:
corner.corner(corner_data, range=[[-11, -10], [-3, 3], [1, 200], [-18, -14]]);

In [ ]:
p_mcmc = flat_chain.mean(0)

#### Nelder-Mead

In [ ]:
def obj_func(params, x, data):
    return -lnprob(params, x, data)

opt_res = minimize(obj_func, p0, args=(x, data), method='Nelder-Mead', options={'disp':True})
p_nm = opt_res.x

#### LM Methods

In [ ]:
p_lm, cov = curve_fit(truncated_powerlaw, x, data, p0=p0, sigma= data * 0.05 + noise)

In [ ]:
f, ax = plt.subplots()
ax.loglog(x / au, data, label='data')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_mcmc), label='MCMC')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_nm), label='NM')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_lm), label='LM')
ax.set_ylim(1e-16, 1e-10)
ax.legend();